# EOP Visualization

In [21]:
import numpy as np
import geopandas as gpd
import plotly.graph_objects as go

In [22]:
# Load country GeoJSON file
gdf = gpd.read_file("countries.geo.json")
gdf

,id,name,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.8554, 20.46318 41.51509,..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.2455, 51.75744 24.29407,..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.5 -55.2, -66.45 -55.25, -6..."
...,...,...,...
175,PSE,West Bank,"POLYGON ((35.54566 32.39399, 35.54525 31.7825,..."
176,YEM,Yemen,"POLYGON ((53.10857 16.65105, 52.38521 16.38241..."
177,ZAF,South Africa,"POLYGON ((31.521 -29.25739, 31.32556 -29.40198..."
178,ZMB,Zambia,"POLYGON ((32.75938 -9.2306, 33.23139 -9.67672,..."


In [23]:


def geo_to_cartesian(lon, lat, radius=1.0):
    lon = np.radians(lon)
    lat = np.radians(lat)
    x = radius * np.cos(lat) * np.cos(lon)
    y = radius * np.cos(lat) * np.sin(lon)
    z = radius * np.sin(lat)
    return x, y, z

trace_list = []

for _, row in gdf.iterrows():
    geom = row['geometry']

    if geom.geom_type == 'Polygon':
        polygons = [geom]
    elif geom.geom_type == 'MultiPolygon':
        polygons = geom.geoms
    else:
        continue

    for poly in polygons:
        coords = np.array(poly.exterior.coords)
        lon, lat = coords[:, 0], coords[:, 1]
        x, y, z = geo_to_cartesian(lon, lat)
        
        trace_list.append(go.Scatter3d(x=x, y=y, z=z,
                                       mode='lines',
                                       line=dict(color='magenta', width=2),
                                       showlegend=False ))

# Adding a sphere for visual context of Earth
phi = np.linspace(0, 2 * np.pi, 100)            # 1D Numpy Array of longitude angle, phi, 0-360 degrees, linearly spaced
theta = np.linspace(0, np.pi, 100)              # 1D Numpy Array Latitude angle, theta, 0-180 degrees, linearly spaced

# Generate 2D grids of x, y, z coordinates on the sphere surface
# np.outer is used to compute all pairwise combinations of spherical components without explicit meshgrid
x = np.outer(np.cos(phi), np.sin(theta))        # x[i,j] = cos(phi[i]) * sin(theta[j])
y = np.outer(np.sin(phi), np.sin(theta))        # y[i,j] = sin(phi[i]) * sin(theta[j])
z = np.outer(np.ones_like(phi), np.cos(theta))  # z[i,j] = cos(theta[j]), broadcasted across phi


surface = go.Surface(x=x, y=y, z=z,
                     opacity=0.85,
                     showscale=False,
                     colorscale=[[0, '#003366'], [1, '#3399FF']])

fig = go.Figure(data=[surface] + trace_list)
fig.update_layout(scene=dict(xaxis=dict(visible=False),
                             yaxis=dict(visible=False),
                             zaxis=dict(visible=False),
                             aspectmode='data',
                             bgcolor='black'),
                margin=dict(t=0, r=0, l=0, b=0))

fig.show()
